視覺化神經網路讓我們更好地瞭解在這個神秘的巨大網絡中發生了什麼。除了這個應用，Leon Gatys和他的合作者有一個非常有趣的工作，被稱為“神經藝術風格”就是利用神經表徵分離與重組的內容和風格的任意影像算灋，提供了一個藝術形象的創作神經算灋。  
結果表明，不同的篩檢程式響應之間的相關性是樣式的表示。很迷人，對吧？

In [1]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import IPython.display as ipyd
import scipy.misc
from libs import utils

/Users/unahsu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Defining Content Features and Style Features

Content features:  
內容的影像內容特徵的供給內容的影像到神經網路的計算和選取這些content_layers啟動。  
Style features:  
對於樣式特徵，我們選取樣式影像層的特徵的相關性（矩陣）。通過新增多個層的特徵相關性，我們獲得了輸入影像的多尺度表示，該影像捕獲了紋理資訊，而不是輸入影像中的對象排列。  
給定的內容特點和風格特點，我們可以設計一個損失函數，使最終的影像包含的內容，但在風格形象風格的插圖。

### Defining the Loss  
Our goal is to create an output image which is synthesized by finding an image that simultaneously matches the content features of the photograph and the style features of the respective piece of art. How can we do that? We can define the loss function as the composition of:  
1. The dissimilarity of the content features between the output image and the content image; and
2. The dissimilarity of the style features between the output image and the style image to the loss function.
  
The following figure gives a very good visualization of the process:

我們從一個嘈雜的初始影像，然後將它作為tensorflow變數，而不是做梯度下降的權重，我們固定的重量和做梯度下降的初始影像來最小化損失函數（這是風格和內容的損失損失的總和）。

通過程式碼理解可能更容易。讓我們先從一些偉大的藝術家準備我們最喜歡的內容形象和風格形象。讓我們繼續用神奇的女人作為內容形象，因為她太棒了！對於風格形象，讓我們用梵古的經典作品《星夜》。

In [3]:
import os
content_directory = 'contents/'
style_directory = 'styles/'

# This is the directory to store the final stylized images
output_directory = 'image_output/'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
# This is the directory to store the half-done images during the training.
checkpoint_directory = 'checkpoint_output/'
if not os.path.exists(checkpoint_directory):
    os.makedirs(checkpoint_directory)
    
content_path = os.path.join(content_directory, 'wonder-woman.jpg')
style_path = os.path.join(style_directory, 'starry-night.jpg')
output_path = os.path.join(output_directory, 'wonder-woman-starry-night-iteration-1000.jpg')

# please notice that the checkpoint_images_path has to contain %s in the file_name
checkpoint_path = os.path.join(checkpoint_directory, 'wonder-woman-starry-night-iteration-1000-%s.jpg')

In [4]:
content_image = utils.imread(content_path)

# You can pass several style images as a list, but let's use just one for now.
style_images = [utils.imread(style_path)]

### Utility functions for loading the convolution layers of VGG19 model

In [5]:
import tensorflow as tf
import numpy as np
import scipy.io
import os

VGG_MEAN = [103.939, 116.779, 123.68]

VGG19_LAYERS = (
    'conv1_1', 'relu1_1', 'conv1_2', 'relu1_2', 'pool1',

    'conv2_1', 'relu2_1', 'conv2_2', 'relu2_2', 'pool2',


    'conv3_1', 'relu3_1', 'conv3_2', 'relu3_2', 'conv3_3',
    'relu3_3', 'conv3_4', 'relu3_4', 'pool3',

    'conv4_1', 'relu4_1', 'conv4_2', 'relu4_2', 'conv4_3',
    'relu4_3', 'conv4_4', 'relu4_4', 'pool4',

    'conv5_1', 'relu5_1', 'conv5_2', 'relu5_2', 'conv5_3',
    'relu5_3', 'conv5_4', 'relu5_4'
)


def net_preloaded(input_image, pooling):
    data_dict = np.load('libs/vgg19.npy', encoding='latin1').item()
    net = {}
    current = input_image
    for i, name in enumerate(VGG19_LAYERS):
        kind = name[:4]
        if kind == 'conv':
            kernels = get_conv_filter(data_dict, name)
            # kernels = np.transpose(kernels, (1, 0, 2, 3))

            bias = get_bias(data_dict, name)
            # matconvnet: weights are [width, height, in_channels, out_channels]
            # tensorflow: weights are [height, width, in_channels, out_channels]

            # bias = bias.reshape(-1)
            current = conv_layer(current, kernels, bias)
        elif kind == 'relu':
            current = tf.nn.relu(current)
        elif kind == 'pool':
            current = pool_layer(current, pooling)

        net[name] = current

    assert len(net) == len(VGG19_LAYERS)
    return net

def conv_layer(input, weights, bias):
    conv = tf.nn.conv2d(input, weights, strides=(1, 1, 1, 1), padding='SAME')
    return tf.nn.bias_add(conv, bias)


def pool_layer(input, pooling):
    if pooling == 'avg':
        return tf.nn.avg_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
                              padding='SAME')
    else:
        return tf.nn.max_pool(input, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1),
                              padding='SAME')

# before we feed the image into the network, we preprocess it by 
# extracting the mean_pixel from it.
def preprocess(image):
    return image - VGG_MEAN

# remember to unprocess it before you plot it out and save it.
def unprocess(image):
    return image + VGG_MEAN

def get_conv_filter(data_dict, name):
    return tf.constant(data_dict[name][0], name="filter")

def get_bias(data_dict, name):
    return tf.constant(data_dict[name][1], name="biases")

### The Algorithm  
This is the main algorithm we will be using to stylize the network. There are a lot of hyper-parameters you can tune. The output image will be stored at output_path, and the checkpoint image (stylized images on every checkpoint_iterations steps) will be stored at checkpoint_path if specified.

In [6]:
import tensorflow as tf
import numpy as np
from functools import reduce
from PIL import Image

# feel free to try different layers
CONTENT_LAYERS = ('relu4_2', 'relu5_2')
STYLE_LAYERS = ('relu1_1', 'relu2_1', 'relu3_1', 'relu4_1', 'relu5_1')

VGG_MEAN = [103.939, 116.779, 123.68]

def stylize(content, styles, network_path='libs/imagenet-vgg-verydeep-19.mat', 
            iterations=1000, content_weight=5e0, content_weight_blend=0.5, style_weight=5e2, 
            style_layer_weight_exp=1,style_blend_weights=None, tv_weight=100,
            learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, pooling='avg',
            print_iterations=100, checkpoint_iterations=100, checkpoint_path=None,
            output_path=None):


    shape = (1,) + content.shape                             #content image shape : (1,433,770,3)
    style_shapes = [(1,) + style.shape for style in styles]  #style image shape : (1,600,800,3)
    content_features = {}
    style_features = [{} for _ in styles]

    
    # scale the importance of each style layers according to their depth. 
    # (deeper layers are more important if style_layers_weights > 1 (default = 1))
    layer_weight = 1.0
    style_layers_weights = {}                                # weight for different network layers
    for style_layer in STYLE_LAYERS:                                    
        style_layers_weights[style_layer] = layer_weight       #'relu1_1','relu2_1',...,'relu5_1'
        layer_weight *= style_layer_weight_exp                 # 1.0


    # normalize style layer weights
    layer_weights_sum = 0
    for style_layer in STYLE_LAYERS:                         #'relu1_1',..., 'relu5_1'
        layer_weights_sum += style_layers_weights[style_layer] # 5.0
    for style_layer in STYLE_LAYERS:
        style_layers_weights[style_layer] /= layer_weights_sum

        
    # FEATURE MAPS FROM CONTENT IMAGE
    # compute the feature map of the content image by feeding it into the network
    # the output net contains the features of each content layer
    g = tf.Graph()
    with g.as_default(), tf.Session() as sess:
        image = tf.placeholder('float', shape=shape)

        net = net_preloaded(image, pooling)             # {'conv1_1':Tensor,relu1_1:Tensor...}
        content_pre = np.array([preprocess(content)])   # (1,433,770,3) subtract the mean pixel
        for layer in CONTENT_LAYERS:                    #'relu4_2', 'relu5_2'
            content_features[layer] = net[layer].eval(feed_dict={image: content_pre})  

            
    # FEATURE MAPS (GRAM MATRICES) FROM STYLE IMAGE
    # compute style features of the style image by feeding it into the network
    # and calculate the gram matrix
    for i in range(len(styles)):
        g = tf.Graph()
        with g.as_default(), tf.Session() as sess:
            image = tf.placeholder('float', shape=style_shapes[i])
            net = net_preloaded(image, pooling)                           
            style_pre = np.array([preprocess(styles[i])])
            for layer in STYLE_LAYERS:              #'relu1_1', 'relu2_1',..., 'relu5_1'
                features = net[layer].eval(feed_dict={image: style_pre})  # relu_1:(1,600,800,64)
                features = np.reshape(features, (-1, features.shape[3]))  # (480000, 64)
                gram = np.matmul(features.T, features) / features.size    # (64,64)
                style_features[i][layer] = gram

                
    # make stylized image using backpropogation
    with tf.Graph().as_default():

        # Generate a random image (the output image) with the same shape as the content image
        initial = tf.random_normal(shape) * 0.256  
        image = tf.Variable(initial)
        net = net_preloaded(image, pooling)


        # CONTENT LOSS
        # we can adjust the weight of each content layers
        # content_weight_blend is the ratio of two used content layers in this example
        content_layers_weights = {}
        content_layers_weights['relu4_2'] = content_weight_blend 
        content_layers_weights['relu5_2'] = 1.0 - content_weight_blend      

        content_loss = 0
        content_losses = []
        for content_layer in CONTENT_LAYERS:
            # Use MSE as content losses
            # content weight is the coefficient for content loss
            content_losses.append(content_layers_weights[content_layer] * content_weight * 
                                  (2 * tf.nn.l2_loss(net[content_layer] - content_features[content_layer]) /
                                   content_features[content_layer].size))
        content_loss += reduce(tf.add, content_losses)



        # STYLE LOSS
        # We can specify different weight for different style images
        # style_layers_weights => weight for different network layers
        # style_blend_weights => weight between different style images

        if style_blend_weights is None:
            style_blend_weights = [1.0/len(style_images) for _ in style_images]
        else:
            total_blend_weight = sum(style_blend_weights)
            # normalization
            style_blend_weights = [weight/total_blend_weight
                                   for weight in style_blend_weights]


        style_loss = 0
        # iterate to calculate style loss with multiple style images
        for i in range(len(styles)):
            style_losses = []
            for style_layer in STYLE_LAYERS:             # e.g. relu1_1
                layer = net[style_layer]                   # relu1_1 of output image:(1,433,770,64)
                _, height, width, number = map(lambda i: i.value, layer.get_shape())  
                size = height * width * number
                feats = tf.reshape(layer, (-1, number))    # (333410,64)

                # Gram matrix for the features in relu1_1 of the output image.
                gram = tf.matmul(tf.transpose(feats), feats) / size
                # Gram matrix for the features in relu1_1 of the style image
                style_gram = style_features[i][style_layer]   

                # Style loss is the MSE for the difference of the 2 Gram matrices
                style_losses.append(style_layers_weights[style_layer] * 2 * 
                                    tf.nn.l2_loss(gram - style_gram) / style_gram.size)
            style_loss += style_weight * style_blend_weights[i] * reduce(tf.add, style_losses)


        # TOTAL VARIATION LOSS  
        # Total variation denoising to do smoothing; 
        # cost to penalize neighboring pixel not used by the original paper by Gatys et al
        # According to the paper Mahendran, Aravindh, and Andrea Vedaldi. 
        # "Understanding deep image representations by inverting them."
        # Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition. 2015.
        tv_y_size = _tensor_size(image[:,1:,:,:])
        tv_x_size = _tensor_size(image[:,:,1:,:])
        tv_loss = tv_weight * 2 * (
            (tf.nn.l2_loss(image[:,1:,:,:] - image[:,:shape[1]-1,:,:]) /
             tv_y_size) +
            (tf.nn.l2_loss(image[:,:,1:,:] - image[:,:,:shape[2]-1,:]) /
             tv_x_size))


        #OVERALL LOSS
        loss = content_loss + style_loss + tv_loss

        train_step = tf.train.AdamOptimizer(learning_rate, beta1, beta2, epsilon).minimize(loss)

        def print_progress():
            print('     iteration: %d\n' % i)
            print('  content loss: %g\n' % content_loss.eval())
            print('    style loss: %g\n' % style_loss.eval())
            print('       tv loss: %g\n' % tv_loss.eval())
            print('    total loss: %g\n' % loss.eval())

        def imsave(path, img):
            img = np.clip(img, 0, 255).astype(np.uint8)
            Image.fromarray(img).save(path, quality=95)

    
    
        # TRAINING
        best_loss = float('inf')
        best = None

        with tf.Session() as sess:

            sess.run(tf.global_variables_initializer())

            if (print_iterations and print_iterations != 0):
                print_progress()

            for i in range(iterations):

                train_step.run()

                last_step = (i == iterations - 1)
                if last_step or (print_iterations and i % print_iterations == 0):
                    print_progress()

                # store output and checkpoint images
                if (checkpoint_iterations and i % checkpoint_iterations == 0) or last_step:
                    this_loss = loss.eval()
                    if this_loss < best_loss:
                        best_loss = this_loss
                        best = image.eval()

                    img_out = unprocess(best.reshape(shape[1:]))

                    output_file = None
                    if not last_step:
                        if checkpoint_path:
                            output_file = checkpoint_path % i
                        else:
                            output_file = output_path

                        if output_file:
                            imsave(output_file, img_out)

    print("finish stylizing.")



def _tensor_size(tensor):
    from operator import mul
    return reduce(mul, (d.value for d in tensor.get_shape()), 1)

In [7]:
checkpoint_path=None
output_path='image_output/wonder-woman-starry-night-tvweight-100.jpg'

stylize(content_image, style_images, iterations=1000,
        content_weight=5e0, content_weight_blend=1, style_weight=5e2, 
        style_layer_weight_exp=1, style_blend_weights=None, tv_weight=100,
        learning_rate=1e1, beta1=0.9, beta2=0.999, epsilon=1e-08, pooling='avg',
        print_iterations=100, checkpoint_iterations=100, checkpoint_path=checkpoint_path,
        output_path=output_path)

     iteration: 0

  content loss: 383717

    style loss: 1.5164e+06

       tv loss: 26.2192

    total loss: 1.90014e+06

     iteration: 0

  content loss: 366404

    style loss: 1.27638e+06

       tv loss: 15739.4

    total loss: 1.65852e+06

     iteration: 100

  content loss: 85520.8

    style loss: 26423.2

       tv loss: 31910.4

    total loss: 143854

     iteration: 200

  content loss: 77676.3

    style loss: 25852

       tv loss: 29432.2

    total loss: 132961

     iteration: 300

  content loss: 77106.3

    style loss: 25445.8

       tv loss: 28503.2

    total loss: 131055

     iteration: 400

  content loss: 78006.9

    style loss: 26281.6

       tv loss: 28695

    total loss: 132983

     iteration: 500

  content loss: 75787.7

    style loss: 26893.5

       tv loss: 28061.2

    total loss: 130742

     iteration: 600

  content loss: 75734.1

    style loss: 25568

       tv loss: 28473.8

    total loss: 129776

     iteration: 700

  content loss